# Prepare the real-time scoring model

The team at Woodgrove Bank has provided you with exported CSV copies of historical data for you to train your model against. Run the following cell to load required libraries and download the data sets from the Azure ML datastore.

In [2]:
#!pip install --upgrade azureml-train-automl-runtime==1.36.0
#!pip install --upgrade azureml-automl-runtime==1.36.0
#!pip install --upgrade scikit-learn
#!pip install --upgrade numpy

StatementMeta(28089669-bbd5-433b-b328-2aaf792d2d3a, 3, 7, Finished, Available)

In [3]:
from azureml.core import Workspace, Environment, Datastore, Dataset
from azureml.core.experiment import Experiment
from azureml.core.run import Run
from azureml.core.model import Model

from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler

# sklearn.externals.joblib was deprecated in 0.21
from sklearn import __version__ as sklearnver
from packaging.version import Version
if Version(sklearnver) < Version("0.21.0"):
    from sklearn.externals import joblib
else:
    import joblib

import numpy as np
import pandas as pd

ws = Workspace.get(subscription_id= "831359a3-06f6-4ed7-a3d0-535b8b673781",
                    resource_group= "My_resourcegroup",
                    name= "Innovative_fraud_detection")

# Load data
ds = Datastore.get(ws, "fd_datastore")

account_ds = Dataset.Tabular.from_delimited_files(path = [(ds, 'transactions/Account_Info.csv')])
fraud_ds = Dataset.Tabular.from_delimited_files(path = [(ds, 'transactions/Fraud_Transactions.csv')])
untagged_ds = Dataset.Tabular.from_delimited_files(path = [(ds, 'transactions/Untagged_Transactions.csv')])

# Create pandas dataframes from datasets
account_df = account_ds.to_pandas_dataframe()
fraud_df = fraud_ds.to_pandas_dataframe()
untagged_df = untagged_ds.to_pandas_dataframe()

StatementMeta(28089669-bbd5-433b-b328-2aaf792d2d3a, 3, 8, Finished, Available)

Performing interactive authentication. Please follow the instructions on the terminal.
To sign in, use a web browser to open the page https://microsoft.com/devicelogin and enter the code EJFA9963H to authenticate.
You have logged in. Now let us find all the subscriptions to which you have access...
Interactive authentication successfully completed.


View the fraud dataframe. 

NOTE: The schema documentation for the data used here is available at:
https://microsoft.github.io/r-server-fraud-detection/input_data.html

In [4]:
fraud_df

StatementMeta(28089669-bbd5-433b-b328-2aaf792d2d3a, 3, 9, Finished, Available)

,transactionID,accountID,transactionAmount,transactionCurrencyCode,transactionDate,transactionTime,localHour,transactionDeviceId,transactionIPaddress
0,65020E58-781D-4FFC-BEF2-0FDF87BE671D,A985156981092344,1148.60,CAD,20130402,14450,20,NA,66.460
1,8EC10EBC-F4BB-4148-9073-4B2BA93C9B34,A985156981066925,150.31,USD,20130403,135015,8,NA,67.800
2,CD624353-E473-4EE0-8BDF-A818627AA1D1,A985156970845915,99.98,USD,20130403,161950,11,NA,96.570
3,C125DD40-6BE3-40E2-8604-5B9540751467,A1688853657886190,53.99,USD,20130403,175759,12,NA,74.680
4,FF74DFD9-5015-440B-97A1-BDAB4A95B272,A985156971082067,119.98,USD,20130403,181256,13,NA,184.340
...,...,...,...,...,...,...,...,...,...
8635,2BC88AB8-2773-4DE8-86D7-E7281C4BF115,A1829582472627460,105.99,USD,20130929,104211,2,NA,99.420
8636,F560D6BE-BCA9-4582-939B-E9274DA743D4,A844427656367602,86.95,USD,20130929,153500,7,NA,209.166
8637,D8EA18C5-0AE8-4054-B380-DED726C64517,A844427695849709,139.00,EUR,20130929,222727,23,NA,94.216
8638,C4D00819-C09F-4A64-AB97-C644865F4412,A1055520904938410,141.39,USD,20130929,233033,18,NA,69.115


View the account info dataframe.

In [5]:
account_df

StatementMeta(28089669-bbd5-433b-b328-2aaf792d2d3a, 3, 10, Finished, Available)

,accountID,transactionDate,transactionTime,accountOwnerName,accountAddress,accountPostalCode,accountCity,accountState,accountCountry,accountOpenDate,accountAge,isUserRegistered,paymentInstrumentAgeInAccount,numPaymentRejects1dPerUser
0,A1688852564389340,20130401,2932,None,None,30170-000,None,MG,BR,None,1.0,False,0.000694,0.0
1,A985156162171434,20130401,3005,None,None,da11 9ps,None,England,GB,None,61.0,True,55.490972,0.0
2,A844427191626038,20130401,12302,None,None,4671,None,Queensland,AU,None,1.0,False,0.002083,0.0
3,A362015581825370,20130401,12446,None,None,-,None,None,PE,None,1.0,False,0.000000,0.0
4,A1829581577560200,20130401,13308,None,None,3780000,None,chillan,CL,None,1.0,False,0.000000,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
199510,A985156194167109,20130610,192749,None,None,43214,None,OH,US,None,1.0,False,0.000000,0.0
199511,A1055520550223960,20130610,191723,None,None,45218,None,OH,US,None,2000.0,True,41.991667,0.0
199512,A1055520426331540,20130610,192004,None,None,92691,None,CA,US,None,84.0,False,83.155556,0.0
199513,A1055521418207590,20130610,192129,None,None,49682,None,MI,US,None,1.0,False,0.002083,0.0


View the untagged transactions dataframe.

In [6]:
###### Reorder the column of dataframe by ascending order in pandas 
cols=untagged_df.columns.tolist()
cols.sort()
untagged_df=untagged_df[cols]

untagged_df

StatementMeta(28089669-bbd5-433b-b328-2aaf792d2d3a, 3, 11, Finished, Available)

,accountID,browserLanguage,browserType,cardNumberInputMethod,cardType,cvvVerifyResult,digitalItemCount,ipCountryCode,ipPostcode,ipState,...,transactionCurrencyConversionRate,transactionDate,transactionDeviceId,transactionDeviceType,transactionID,transactionIPaddress,transactionMethod,transactionScenario,transactionTime,transactionType
0,A985156985579195,en-AU,None,None,VISA,M,1,au,3000,victoria,...,None,20130409,None,None,5EAC1EBD-1428-4593-898E-F4B56BC3FA06,121.219,None,A,95040,P
1,A985156966855837,en-AU,None,None,VISA,M,0,us,14534,new york,...,None,20130409,None,None,48C88D1C-3705-472B-A4A3-5FCE45A5429B,216.150,None,A,94256,P
2,A844428012992486,nn-NO,None,None,MC,M,1,no,1006,oslo,...,None,20130409,None,None,13B2A110-EA04-42CD-88CC-A85814A5C961,94.246,None,A,95257,P
3,A1055521358474530,en-US,None,None,AMEX,M,0,ae,0,dubayy,...,None,20130409,None,None,C34F7C20-6203-42F5-A41B-AF26177345BE,92.970,None,A,102958,P
4,A844428033864668,it-IT,None,None,VISA,U,1,it,39100,bolzano,...,None,20130409,None,None,C78542E6-0951-4B63-B420-BEF750B98BCD,95.229,None,A,103514,P
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
199995,A844427716212595,en-US,None,None,VISA,M,1,us,33144,florida,...,None,20130927,None,None,4B42DDD2-B325-4153-A274-F9CC0A14F56F,162.229,None,A,195831,P
199996,A844428106261845,fr-FR,None,None,None,None,1,fr,36000,indre,...,None,20130927,None,None,4BFD3DE4-1474-4F57-ACB0-8B8861095496,213.245,None,A,200022,P
199997,A914800203072004,en-US,None,None,AMEX,M,0,us,14534,new york,...,None,20130927,None,None,4178D1BF-66E6-4449-A491-1C3750531DE4,216.150,None,A,195832,P
199998,A985156680378469,ru-RU,None,None,MC,M,1,ru,100000,moskva,...,None,20130927,None,None,415575D3-9931-4D70-8A4E-3D7E12BA2F73,95.165,None,A,200859,P


## Prepare data

The raw data has some issues we need to cleanup before we can use it to train a model, which we perform in the following cells.

### Prepare accounts

Begin by cleaning the data in accounts data set.
Remove columns that have very few or no values: `accountOwnerName`, `accountAddress`, `accountCity` and `accountOpenDate` 

In [7]:
account_df_clean = account_df[["accountID", "transactionDate", "transactionTime", 
                               "accountPostalCode", "accountState", "accountCountry", 
                               "accountAge", "isUserRegistered", "paymentInstrumentAgeInAccount", 
                               "numPaymentRejects1dPerUser"]]

StatementMeta(28089669-bbd5-433b-b328-2aaf792d2d3a, 3, 12, Finished, Available)

Create a copy of the dataframe so our data manipulation does not affect the original.

In [8]:
account_df_clean = account_df_clean.copy()

StatementMeta(28089669-bbd5-433b-b328-2aaf792d2d3a, 3, 13, Finished, Available)

Let's ensure that values that are not numeric (e.g., they have incorrect string values or garbage data) are converted to NaN and then we can fill those NaN values with 0.

In [9]:
account_df_clean['paymentInstrumentAgeInAccount'] = pd.to_numeric(account_df_clean['paymentInstrumentAgeInAccount'], errors='coerce')
account_df_clean['paymentInstrumentAgeInAccount'] = account_df_clean[['paymentInstrumentAgeInAccount']].fillna(0)['paymentInstrumentAgeInAccount']

StatementMeta(28089669-bbd5-433b-b328-2aaf792d2d3a, 3, 14, Finished, Available)

Next, let's convert the `numPaymentRejects1dPerUser` so that the column has a datatype of `float` instead of `object`.

In [10]:
account_df_clean["numPaymentRejects1dPerUser"] = account_df_clean[["numPaymentRejects1dPerUser"]].astype(float)["numPaymentRejects1dPerUser"]

StatementMeta(28089669-bbd5-433b-b328-2aaf792d2d3a, 3, 15, Finished, Available)

Let's take a look at the results of our cleanup of this one column. Looks like the most payment declines/rejects that happen to a given user in one day happens either zero times or 1 time, and then trails off quickly. After 5 times the number of rejects per user per day is down to 136.

In [11]:
account_df_clean["numPaymentRejects1dPerUser"].value_counts()

StatementMeta(28089669-bbd5-433b-b328-2aaf792d2d3a, 3, 16, Finished, Available)

0.0     191382
1.0       5500
2.0       1476
3.0        562
4.0        254
5.0        136
6.0         51
7.0         30
8.0         27
10.0        24
9.0         14
17.0         9
14.0         6
13.0         4
16.0         3
12.0         2
32.0         2
11.0         2
18.0         1
28.0         1
29.0         1
15.0         1
26.0         1
23.0         1
Name: numPaymentRejects1dPerUser, dtype: int64

`account_df_clean` is now ready for use in modeling.

### Prepare untagged transactions

Next, cleanup the untagged transactions data set. There are 16 columns in the untagged_transactions whose values are all null, let's drop these columns to simplify our dataset.

In [12]:
untagged_df_clean = untagged_df.dropna(axis=1, how="all").copy()

StatementMeta(28089669-bbd5-433b-b328-2aaf792d2d3a, 3, 17, Finished, Available)

We can examine the count of non-null values, and view the inferred data type for each column by running the following cell. Looking at the output of the cell, we have some work to do. For a start, we have columns with fewer than 200,000 non-null values. This means there are some null values in that column that we need to fix.

Let's cleanup the `localHour` field. 

Replace null values in `localHour` with `-99`. Also replace values of `-1` with `-99`.

In [13]:
untagged_df_clean["localHour"] = untagged_df_clean["localHour"].fillna(-99)
untagged_df_clean.loc[untagged_df_clean.loc[:,"localHour"] == -1, "localHour"] = -99

StatementMeta(28089669-bbd5-433b-b328-2aaf792d2d3a, 3, 18, Finished, Available)

Confirm the values now look good.

In [14]:
untagged_df_clean["localHour"].value_counts()

StatementMeta(28089669-bbd5-433b-b328-2aaf792d2d3a, 3, 19, Finished, Available)

 13.0    12783
 15.0    12720
 14.0    12694
 10.0    12439
 11.0    12372
 12.0    12315
 16.0    11929
 19.0    11880
 20.0    11588
 18.0    11539
 17.0    11458
 9.0     11200
 21.0     9728
 8.0      8768
-99.0     7037
 22.0     6986
 7.0      5368
 23.0     4716
 6.0      3094
 0.0      2944
 1.0      1859
 5.0      1596
 2.0      1122
 4.0       969
 3.0       896
Name: localHour, dtype: int64

Clean up the remaining null fields:
- Fix missing values for location fields by setting them to `NA` for unknown. 
- Set `isProxyIP` to False
- Set `cardType` to `U` for unknown (which is a new level)
- Set `cvvVerifyResult` to `N` which means for those where the transaction failed because the wrong CVV2 number was entered ro no CVV2 numebr was entered, treat those as if there was no CVV2 match.

In [15]:
untagged_df_clean = untagged_df_clean.fillna(value={"ipState": "NA", "ipPostcode": "NA", "ipCountryCode": "NA", 
                               "isProxyIP":False, "cardType": "U", 
                               "paymentBillingPostalCode" : "NA", "paymentBillingState":"NA",
                               "paymentBillingCountryCode" : "NA", "cvvVerifyResult": "N"
                              })

StatementMeta(28089669-bbd5-433b-b328-2aaf792d2d3a, 3, 20, Finished, Available)

Confirm all null values have been addressed.

The `transactionScenario` column provides no insights because all rows have the same `A` value. Let's drop that column. Same idea for the `transactionType` column.

In [16]:
del untagged_df_clean["transactionScenario"]

StatementMeta(28089669-bbd5-433b-b328-2aaf792d2d3a, 3, 21, Finished, Available)

In [17]:
del untagged_df_clean["transactionType"]

StatementMeta(28089669-bbd5-433b-b328-2aaf792d2d3a, 3, 22, Finished, Available)

`untagged_df_clean` is now ready for use in modeling.

### Prepare fraud transactions

Now move on to preparing the fraud transactions data set.

The `transactionDeviceId` has no meaningful values, so we will drop it.

In [18]:
fraud_df_clean = fraud_df.copy()
del fraud_df_clean['transactionDeviceId']

StatementMeta(28089669-bbd5-433b-b328-2aaf792d2d3a, 3, 23, Finished, Available)

The fraud data set has a `localHour` field that we need to fill missing values, just as we did for the account data set.

In [19]:
fraud_df_clean["localHour"] = fraud_df_clean["localHour"].fillna(-99)

StatementMeta(28089669-bbd5-433b-b328-2aaf792d2d3a, 3, 24, Finished, Available)

Examine your work, you should have 8640 non-null values in each column.

`fraud_df_clean` is now ready for use in modeling.

## Create labels

The goal is to create a dataframe with all transactions, where each transaction is tagged via the `isFraud` column with a value of `0` - no fraud or `1` - fraudulent. 

Any transactions that appear in untagged_transactions dataframe that also appear in the fraud dataframe will be marked as fraudulent. 

The remaining transactions will be marked as not fraudulent. 

Run the following cells to create the labels series.

In [20]:
all_labels = untagged_df_clean["transactionID"].isin(fraud_df_clean["transactionID"])

StatementMeta(28089669-bbd5-433b-b328-2aaf792d2d3a, 3, 25, Finished, Available)

In [21]:
all_transactions = untagged_df_clean

StatementMeta(28089669-bbd5-433b-b328-2aaf792d2d3a, 3, 26, Finished, Available)

## Create feature engineering pipeline

In the following cells we will define two custom estimators that will be used in pipeline to prepare the data.

We collect these estimators in a module and then save this module file in the models directory. Then we will use the classes in this module during both model training and model scoring. During deployment, when you register the model using `Model.register` (as we do later in `deployModelAsWebService`), all files in the models directory are uploaded with the model.

The module containing the estimators used to transform the data before scoring needs to be deployed with the model. This ensures that the code can be executed in the context of the webservice, when the serialized pipeline is loaded and used for scoring. 

First we need to create the models directory if it does not already exist.

In [22]:
import uuid
import os

# Create a temporary folder to store locally relevant content for this notebook
tempFolderName = 'FileStore/mcw_cdb_{0}'.format(uuid.uuid4())

models_dir = tempFolderName + "/models/"
if not os.path.exists(models_dir):
    os.makedirs(models_dir)

StatementMeta(28089669-bbd5-433b-b328-2aaf792d2d3a, 3, 27, Finished, Available)

Then we can save our estimators module.

In [23]:
# write out to models/customestimators.py
scoring_service = """
import pandas as pd
from sklearn.base import BaseEstimator, TransformerMixin
class NumericCleaner(BaseEstimator, TransformerMixin):
    def __init__(self):
        self = self
    def fit(self, X, y=None):
        print("NumericCleaner.fit called")
        return self
    def transform(self, X):
        print("NumericCleaner.transform called")
        X["localHour"] = X["localHour"].fillna(-99)
        X.loc[X.loc[:,"localHour"] == -1, "localHour"] = -99
        return X

class CategoricalCleaner(BaseEstimator, TransformerMixin):
    def __init__(self):
        self = self
    def fit(self, X, y=None):
        print("CategoricalCleaner.fit called")
        return self
    def transform(self, X):
        print("CategoricalCleaner.transform called")
        X = X.fillna(value={"cardType":"U","cvvVerifyResult": "N"})
        return X
""" 

with open(models_dir + "customestimators.py", "w") as file:
    file.write(scoring_service)

StatementMeta(28089669-bbd5-433b-b328-2aaf792d2d3a, 3, 28, Finished, Available)

You need to add the created module to our python search path so it can be found and loaded in this notebook environment:

In [24]:
import sys
from os.path import dirname
sys.path.append(models_dir)

StatementMeta(28089669-bbd5-433b-b328-2aaf792d2d3a, 3, 29, Finished, Available)

Next, load the estimators.

In [25]:
from customestimators import NumericCleaner, CategoricalCleaner

StatementMeta(28089669-bbd5-433b-b328-2aaf792d2d3a, 3, 30, Finished, Available)

Now build the pipeline that will prepare the data. 

The gist of the following cell is to split the data preparation into two paths, splitting the data sets vertically, and then combine the result. The `ColumnTransformer` will effectively concatenate the data frame that results from the numeric transformations with the data frame resulting from the categorical transformations. 

- Numeric Transformer Pipeline: We use the custom transformers created previously to cleanup the numeric columns. Since the model you will train in this notebook is a Support Vector Machine classifier, we need to standardize the scale of numeric values which is what the `StandardScaler` provides.
- Categorical Transformer Pipeline: We use the custome transformer created previously cleanup the categorical columns. Then we one-hot encode each value of each categorical column, resulting in a wider data frame with one column for each possible value (and 1 appearing in rows that had that value).

In [26]:
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler

numeric_features=["transactionAmountUSD", "transactionDate", "transactionTime", "localHour", 
                  "transactionIPaddress", "digitalItemCount", "physicalItemCount"]

categorical_features=["transactionCurrencyCode", "browserLanguage", "paymentInstrumentType", "cardType", "cvvVerifyResult"]                           

numeric_transformer = Pipeline(steps=[
    ('cleaner', NumericCleaner()),
    ('scaler', StandardScaler())
])
                               
categorical_transformer = Pipeline(steps=[
    ('cleaner', CategoricalCleaner()),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ])

StatementMeta(28089669-bbd5-433b-b328-2aaf792d2d3a, 3, 31, Finished, Available)

In [27]:
import sklearn
print(sklearn.__version__)

StatementMeta(28089669-bbd5-433b-b328-2aaf792d2d3a, 3, 32, Finished, Available)

0.23.2


Let's confirm we run all our historical data thru this transformation pipeline and observe the resulting shape.

In [28]:
preprocessed_result = preprocessor.fit_transform(all_transactions)

StatementMeta(28089669-bbd5-433b-b328-2aaf792d2d3a, 3, 33, Finished, Available)

NumericCleaner.fit called
NumericCleaner.transform called
CategoricalCleaner.fit called
CategoricalCleaner.transform called


In [29]:
preprocessed_result.shape

StatementMeta(28089669-bbd5-433b-b328-2aaf792d2d3a, 3, 34, Finished, Available)

(200000, 292)

In [30]:
pd.DataFrame(preprocessed_result.todense())

StatementMeta(28089669-bbd5-433b-b328-2aaf792d2d3a, 3, 35, Finished, Available)

,0,1,2,3,4,5,6,7,8,9,...,282,283,284,285,286,287,288,289,290,291
0,-0.152051,-1.422822,-0.516830,0.417653,0.265748,0.218614,-0.357016,0.0,1.0,0.0,...,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.555361,-1.422822,-0.527675,-5.086697,1.964129,-1.403816,0.593898,0.0,1.0,0.0,...,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
2,-0.039733,-1.422822,-0.513828,-0.002170,-0.216818,0.218614,-0.357016,0.0,0.0,0.0,...,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2.533650,-1.422822,-0.407303,0.184418,-0.239646,-1.403816,3.446637,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.150220,-1.422822,-0.399612,0.044477,-0.199231,0.218614,-0.357016,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
199995,-0.109464,1.551414,0.877384,0.184418,0.999445,0.218614,-0.357016,0.0,0.0,0.0,...,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
199996,-0.148567,1.551414,0.935357,0.510947,1.912156,0.218614,-0.357016,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
199997,0.277925,1.551414,0.877398,0.184418,1.964129,-1.403816,0.593898,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
199998,-0.136933,1.551414,0.946935,0.604241,-0.200376,0.218614,-0.357016,0.0,0.0,0.0,...,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0


## Create pipeline and train a simple model

Now you will build upon the transformation pipeline you created previously to train a model to classify rows as fraudulent or not fraudulent.

Run the following cells to make sure you've imported the dependencies for the pipeline (you probably already have, but having them clearly loaded here will help you when porting your code to a web service).

In [31]:
from customestimators import NumericCleaner, CategoricalCleaner
from sklearn.model_selection import train_test_split

StatementMeta(28089669-bbd5-433b-b328-2aaf792d2d3a, 3, 36, Finished, Available)

As might be obvious, our data has a lot of samples that are not fraudulent. If we proceed to train a model, we will effectively train the model to predict non-fraud. This situation where one class (non-fraud) appears much more often than the others (fraud) is called a class imbalance, and to mitigate its effect we can reduce the number of non-fraud samples so that we have the same number of non-fraud and fraud samples. 

Run the following cells to downsize and then randomly sample 1,151 non-fraud rows, and then we'll union these row with our 1,151 fraud rows.

> Feel free to ignore any `SettingWithCopyWarning` warnings in the cell output below.

In [32]:
only_fraud_samples = all_transactions.loc[all_labels == True]
only_fraud_samples["label"] = True
only_non_fraud_samples = all_transactions.loc[all_labels == False]
only_non_fraud_samples["label"] = False
random_non_fraud_samples = only_non_fraud_samples.sample(n=1151, replace=False, random_state=42)
balanced_transactions = random_non_fraud_samples.append(only_fraud_samples)

balanced_transactions["label"].value_counts()

StatementMeta(28089669-bbd5-433b-b328-2aaf792d2d3a, 3, 37, Finished, Available)

/tmp/ipykernel_25943/2698239665.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  only_fraud_samples["label"] = True
/tmp/ipykernel_25943/2698239665.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  only_non_fraud_samples["label"] = False


False    1151
True     1151
Name: label, dtype: int64

Next, you need to separate out the label column from the dataframe so the labels are not used as input features:

In [33]:
balanced_labels = balanced_transactions["label"]
del balanced_transactions["label"]

StatementMeta(28089669-bbd5-433b-b328-2aaf792d2d3a, 3, 38, Finished, Available)

Now you will create subsets of the training data frame, one that will be used for training the model `X_train` and `y_train` and the another that reserved for testing its performance `X_test` and `y_test`.

In [34]:
X_train, X_test, y_train, y_test = train_test_split(balanced_transactions, balanced_labels, 
                                                    test_size=0.2, random_state=42)

StatementMeta(28089669-bbd5-433b-b328-2aaf792d2d3a, 3, 39, Finished, Available)

Now train the model. In this case, you will use the `LinearSVC` class.

> Feel free to ignore any `ConvergenceWarning` warnings in the cell output below

In [35]:
import numpy as np
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import LinearSVC

svm_clf = Pipeline((
    ("preprocess", preprocessor),
    ("linear_svc", LinearSVC(C=1, loss="hinge"))
))
svm_clf.fit(X_train, y_train)

StatementMeta(28089669-bbd5-433b-b328-2aaf792d2d3a, 3, 40, Finished, Available)

NumericCleaner.fit called
NumericCleaner.transform called
CategoricalCleaner.fit called
CategoricalCleaner.transform called


/home/trusted-service-user/cluster-env/env/lib/python3.8/site-packages/sklearn/svm/_base.py:976: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "


Pipeline(steps=[('preprocess',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('cleaner',
                                                                   NumericCleaner()),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  ['transactionAmountUSD',
                                                   'transactionDate',
                                                   'transactionTime',
                                                   'localHour',
                                                   'transactionIPaddress',
                                                   'digitalItemCount',
                                                   'physicalItemCount']),
                                                 ('cat',
                           

Test the model predicting against a single row from the test set.

In [47]:
import datetime
val = [['A985156985579195', 'en-AU', 'VISA', 'M', 1, 'au', '3000', 'victoria', False, 19, 'AU', '3122', 'Victoria', 'CREDITCARD', 0, 99.0, 103.48965, 'AUD', datetime.date(2013, 4, 9), '5EAC1EBD-1428-4593-898E-F4B56BC3FA06', 121.219, datetime.time(9, 50)]]
test_df = pd.DataFrame(val,columns = desired_cols)

svm_clf.predict()
print(X_test[0:1])

StatementMeta(28089669-bbd5-433b-b328-2aaf792d2d3a, 3, 52, Finished, Available)

TypeError: predict() missing 1 required positional argument: 'X'

Next, evaluate the model by examining how well it is predicting against all data in the training set.

In [37]:
y_train_preds = svm_clf.predict(X_train)

StatementMeta(28089669-bbd5-433b-b328-2aaf792d2d3a, 3, 42, Finished, Available)

NumericCleaner.transform called
CategoricalCleaner.transform called


Use a confusion matrix to see how your model performed when correctly predicting non-fraud and fraud (the top left and bottom right values). Also, examine how your model made mistakes (the bottom left and top right values). In the below, the column headers are predicted non-fraud and predicted fraud, and the row headers are actually non-fraud, and actually fraud (e.g., as described by the training data).

In [38]:
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
confusion_matrix(y_train, y_train_preds)

StatementMeta(28089669-bbd5-433b-b328-2aaf792d2d3a, 3, 43, Finished, Available)

array([[385, 536],
       [112, 808]])

Take a look at the performance of your model using the common set of metrics for a classifier. Do you think this is good or bad?

In [39]:
print("Accuracy:", accuracy_score(y_train, y_train_preds))
print("Precision:", precision_score(y_train, y_train_preds))
print("Recall:", recall_score(y_train, y_train_preds))
print("F1:", f1_score(y_train, y_train_preds))
print("AUC:", roc_auc_score(y_train, y_train_preds))

StatementMeta(28089669-bbd5-433b-b328-2aaf792d2d3a, 3, 44, Finished, Available)

Accuracy: 0.6480173818576861
Precision: 0.6011904761904762
Recall: 0.8782608695652174
F1: 0.7137809187279153
AUC: 0.6481423783222395


Given that this is just a parsimonous model, this model provides a start that performs better than random (as indicated by the AUC being greater than 0.5). There is more work (such as additional feature engineering) that can be done to improve this beyond the current performance that you would want to do before deploying it in production, but that is out of scope for this lab. A parsiminous model helps us to both see if the desired classification is possible given the data and allows to quickly get to something we can deploy as a service to enable integration early on. Then we can iterate deploying improved versions of the model.

Now, evaluate the same using the test data set, using data the trained model has not seen. How does it perform?

In [40]:
y_test_preds = svm_clf.predict(X_test)
print(confusion_matrix(y_test, y_test_preds))
print(accuracy_score(y_test, y_test_preds))
print("Accuracy:", accuracy_score(y_test, y_test_preds))
print("Precision:", precision_score(y_test, y_test_preds))
print("Recall:", recall_score(y_test, y_test_preds))
print("F1:", f1_score(y_test, y_test_preds))
print("AUC:", roc_auc_score(y_test, y_test_preds))

StatementMeta(28089669-bbd5-433b-b328-2aaf792d2d3a, 3, 45, Finished, Available)

NumericCleaner.transform called
CategoricalCleaner.transform called
[[ 98 132]
 [ 34 197]]
0.6399132321041214
Accuracy: 0.6399132321041214
Precision: 0.5987841945288754
Recall: 0.8528138528138528
F1: 0.7035714285714286
AUC: 0.639450404667796


The overall performance of the model against data it has not seen (the test data) is similar to how it performs with the training data. That's a good sign, indicating we did not overfit the model to the training data.

Next, let's look the steps to prepare the model for deployment as a web service.

## Save the model to disk

In preparation for deploying the model, you need to save the model to disk.

In [41]:
joblib.dump(svm_clf, models_dir + 'fraud_score.pkl')

StatementMeta(28089669-bbd5-433b-b328-2aaf792d2d3a, 3, 46, Finished, Available)

['FileStore/mcw_cdb_b973f0d8-09cb-466c-a8ae-e15d3de8b5a7/models/fraud_score.pkl']

## Test loading the model

Next simulate re-loading the model from disk, just like the web service (which you will create in a moment) will have to do.

In [42]:
import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.base import BaseEstimator, TransformerMixin
from customestimators import NumericCleaner, CategoricalCleaner

# sklearn.externals.joblib was deprecated in 0.21
from sklearn import __version__ as sklearnver
from packaging.version import Version
if Version(sklearnver) < Version("0.21.0"):
    from sklearn.externals import joblib
else:
    import joblib

desired_cols = ['accountID',
 'browserLanguage',
 'cardType',
 'cvvVerifyResult',
 'digitalItemCount',
 'ipCountryCode',
 'ipPostcode',
 'ipState',
 'isProxyIP',
 'localHour',
 'paymentBillingCountryCode',
 'paymentBillingPostalCode',
 'paymentBillingState',
 'paymentInstrumentType',
 'physicalItemCount',
 'transactionAmount',
 'transactionAmountUSD',
 'transactionCurrencyCode',
 'transactionDate',
 'transactionID',
 'transactionIPaddress',
 'transactionTime']

scoring_pipeline = joblib.load(models_dir + 'fraud_score.pkl')

StatementMeta(28089669-bbd5-433b-b328-2aaf792d2d3a, 3, 47, Finished, Available)

In [43]:
untagged_ds = Dataset.Tabular.from_delimited_files(path = [(ds, 'transactions/Untagged_Transactions.csv')])
untagged_df_fresh = untagged_ds.to_pandas_dataframe()
untagged_df_fresh=untagged_df_fresh[desired_cols]

test_pipeline_preds = scoring_pipeline.predict(untagged_df_fresh)
test_pipeline_preds

StatementMeta(28089669-bbd5-433b-b328-2aaf792d2d3a, 3, 48, Finished, Available)

NumericCleaner.transform called
CategoricalCleaner.transform called


array([ True,  True, False, ..., False, False, False])

In [44]:
one_row = untagged_df_fresh.iloc[:1]
test_pipeline_preds2 = scoring_pipeline.predict(one_row)
test_pipeline_preds2

StatementMeta(28089669-bbd5-433b-b328-2aaf792d2d3a, 3, 49, Finished, Available)

NumericCleaner.transform called
CategoricalCleaner.transform called


array([ True])

## Register the model in the Azure ML workspace

In [ ]:
import azureml
from azureml.core import Workspace, Webservice
from azureml.core.model import Model
from azureml.exceptions import WebserviceException
from azureml.core.resource_configuration import ResourceConfiguration

StatementMeta(, , , Cancelled, )

In [ ]:
print(sklearnver)

StatementMeta(, , , Cancelled, )

Register the model, providing details on the framework that was used to create the model. The cell output above shows the version of scikit-learn we are using. Additionally, we specify the desired resources (CPU and Memory) to be allocated for the deployment of the model.

In [ ]:
from azureml.core.resource_configuration import ResourceConfiguration

# Register the model with the workspace
registered_model = Model.register(model_path=models_dir, model_name="fraud-score", workspace=ws, model_framework=Model.Framework.SCIKITLEARN, model_framework_version=sklearnver, resource_configuration=ResourceConfiguration(cpu=1, memory_in_gb=0.5), description='Fraud Detection Model')

StatementMeta(, , , Cancelled, )

> **Note**: Please note that executing the next few cells can take between **7** and **10** minutes.

Deploy the model to Azure Container Instances. Once deployment is complete - you will see the message **ACI Service creation operation finished, operation "Succeeded"**

In [ ]:
service_name = "scoringservice"

# delete any existing service with the same name
try:
  Webservice(ws, service_name).delete()
except WebserviceException:
  pass

# deploy the registered model to Azure Container Instances
service = Model.deploy(ws, service_name, [registered_model])
service.wait_for_deployment(show_output=True)

StatementMeta(, , , Cancelled, )